# Grammar Generation

Grammar is generated based on levels from `levels/cleaned_maps/` which are generated from super mario bros levels.

In [1]:
MAP_DIRECTORY = 'levels/cleaned_maps'

We are going to generate 5 grammars, where grammar 1 looks back one column. Grammar 2 looks back two columns and may be:

```
ab -> c
bc -> a,b,d
```

I'm going to diverge from this, though, and apply totals as well. For example, in the case of `ab->c` the weight for `c` would be 1. However, in the other case it would be based on the number of appearances that it made while looking through the maps. Therefore, let's say after `bc`, `a` showed up 10 times, `b` 2 times, and `d` 4 times. This means the weights would be `10/16`, `2/16`, and `4/16` respectively.